In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def IRLS(X, y, w_init, max_iter, eps, lam):
    w = w_init
    for i in range(max_iter):
        w_old = w
        w = w_old - np.linalg.inv(X.T.dot(X) + lam * np.identity(X.shape[1])).dot(X.T).dot(X.dot(w_old) - y)
        if np.linalg.norm(w - w_old) < eps:
            break
    return w

# example
X = np.array([[1, 1], [1, 2], [1, 3], [1, 4], [1, 5]])
y = np.array([1, 2, 3, 4, 5])
w_init = np.array([0, 0])
max_iter = 100
eps = 1e-6
lam = 0.1
w = IRLS(X, y, w_init, max_iter, eps, lam)
print(w)

[3.81553526e-08 9.99999989e-01]
